## (1) Entity feature extraction using spacy

In [1]:
# Input: a sentence
# Output: a list of entities with different types

In [3]:
!pip install spacy

  Using cached blis-0.7.4-cp38-cp38-win_amd64.whl (6.5 MB)
  Using cached murmurhash-1.0.5-cp38-cp38-win_amd64.whl (21 kB)
  Using cached cymem-2.0.5-cp38-cp38-win_amd64.whl (36 kB)
  Using cached wasabi-0.8.2-py3-none-any.whl (23 kB)
  Using cached preshed-3.0.5-cp38-cp38-win_amd64.whl (112 kB)


In [12]:
!python -m spacy download en_core_web_sm

  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.0
    Uninstalling en-core-web-sm-2.2.0:
      Successfully uninstalled en-core-web-sm-2.2.0
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-09-29 13:07:03.236236: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-09-29 13:07:03.236533: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
import spacy
# nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()

entity_types = ['PERSON','NORP','FAC','ORG','GPE','LOC','PRODUCT','EVENT','WORK_OF_ART','LAW',
                'LANGUAGE','DATE','TIME','PERCENT','MONEY','QUANTITY','ORDINAL','CARDINAL']

entity_name = ['person','Nationality','Building','Institution','country','location','PRODUCT','EVENT','Title','LAW',
                'LANGUAGE','DATE','TIME','PERCENT','MONEY','QUANTITY','ORDINAL','CARDINAL']

def entity_extraction(sentence):
    entity_list = []
    doc = nlp(sentence)
    for ent in doc.ents:
        entity_list.append([ent.text, ent.start_char, ent.end_char, ent.label_])
    
    return entity_list
    

In [15]:
sentence = "Apple is looking at buying U.K. startup for $1 billion"

entity_list = entity_extraction(sentence)
print(entity_list)

[['Apple', 0, 5, 'ORG'], ['U.K.', 27, 31, 'GPE'], ['$1 billion', 44, 54, 'MONEY']]


## (2) Bag of Words and TF-IDF features extraction

Please see the following Tutorial

http://dsspace.wzb.eu/pyug/text_proc_feature_extraction/

## (3) Linguistic feature extraction


Please see the following Tutorial on Linguistic Feature Extraction from Text

https://github.com/victoria-ianeva/NERA-Tutorial-on-linguistic-feature-extraction/blob/master/Linguistic%20Features%20NBME%20Items%20NERA%20v2.ipynb

In [16]:
# 3-1 discourse_and_pragmatic_feature

def num_discourse_markers(stanza_doc):
    # Ref: https://universaldependencies.org/docsv1/u/dep/all.html#al-u-dep/discourse
    """Returns the number of discourse markers
        Args:
            stanza_doc (obj): The stanza document object
        Returns
            (int): the number of discourse markers
    """
    return len([1 for word in stanza_doc.sentences[0].words if word.deprel == 'discourse'])

In [17]:
!pip install blabla

  Created wheel for blabla: filename=blabla-0.2.2-py3-none-any.whl size=30905 sha256=f6b0d54c8ca93cd7bb9a706d7797d0a586ac2bf6ae7ee7173672dd593707ccf3
  Stored in directory: c:\users\hhc0025\appdata\local\pip\cache\wheels\84\12\4b\15386930dab17a565fc4929c9418614e4007e88de2471df2a4
Successfully built blabla
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.50.2
    Uninstalling tqdm-4.50.2:


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\tqdm-4.50.2.dist-info\\direct_url.json'
Consider using the `--user` option or check the permissions.



In [19]:
# 3-2 lexico_semantic_feature

from collections import Counter
import math
from blabla.utils.global_params import *
from blabla.utils import *


def num_demonstratives(stanza_doc):
    """The number of demonstravives
            Args:
                stanza_doc (nltk.Tree): The dependency parse tree
            Returns:
                (int): the number of demonstravives
        """
    return len([1 for word in stanza_doc.sentences[0].words if ((word.feats is not None) and ('PronType=Dem' in word.feats))])


def num_unique_words(stanza_doc):
    """Returns the number of unique words
        Args:
            stanza_doc (nltk.Tree): The dependency parse tree
        Returns:
            number of unique words 
    """
    return len(set([word.text for word in stanza_doc.sentences[0].words]))


def num_word_types(stanza_doc):
    """Returns the number of word types
        Args:
            stanza_doc (nltk.Tree): The dependency parse tree
        Returns:
            number of word types
    """
    return len(set([word.lemma for word in stanza_doc.sentences[0].words]))


def compute_mean_word_length(stanza_doc):
    """Returns the mean word length
            Args:
                stanza_doc (nltk.Tree): The dependency parse tree
                
            Returns:
                mean length of all words in the sentence
        """
    return np.mean([len(word.text) for word in stanza_doc.sentences[0].words])

In [20]:
# 3-3 morpho_syntactic_feature

from blabla.utils.global_params import *


def num_inflected_verbs(stanza_doc):
    """Returns the number of inflected verbs
		Args:
			None
		Returns
			num_inflected_verbs (int): The number of inflected verbs
	"""
    num_inflected_verbs = 0
    words = stanza_doc.sentences[0].words
    for word_idx in range(len(words) - 1):
        if (words[word_idx].pos == VERB) and (
            words[word_idx].text != words[word_idx].lemma
        ):
            num_inflected_verbs += 1
    return num_inflected_verbs


def num_gerund_verbs(stanza_doc):
    """The number of gerund verbs
		Args:
			None
		Returns:
			num_gerunds (int): the number of gerund verbs
	"""
    num_gerunds = 0
    for word in stanza_doc.sentences[0].words:
        if word.feats is not None:
            if (word.pos == VERB) and ('VerbForm=Ger' in word.feats):
                num_gerunds += 1
    return num_gerunds


def num_participle_verbs(stanza_doc):
    """The number of participle verbs
		Args:
			None
		Returns:
			num_participle_verbs (int): the number of participle verbs
	"""
    num_participle_verbs = 0
    for word in stanza_doc.sentences[0].words:
        if word.feats is not None:
            if (word.pos == VERB) and (
                'VerbForm=Part' in word.feats
            ):  # what if there are multiple words next to each other with Part?
                num_participle_verbs += 1
    return num_participle_verbs

In [21]:
# 3-4 pos_tag_counting

import stanza


class PosTagCounter(object):
    """The class that counts the number of pos tags of various types in a sentence
    """

    def __init__(self, stanza_doc):
        """The initialization method that take a dependency parse tree as input
            Args:
                stanza_doc (nltk.Tree): the dependency parse tree
            Returns:
                None
        """
        self.stanza_doc = stanza_doc

    def get_pos_tag_count(self, pos_tag):
        """Returns the number of nouns
            Args:
                None
            Returns:
                number of nouns in the sentence
        """
        return len([word for word in self.stanza_doc.sentences[0].words if (word.pos == pos_tag)])

In [22]:
# 3-5 syntactic_feature

from nltk.tree import Tree
from blabla.sentence_processor.yngve_tree import YngveNode
from blabla.sentence_processor.pos_tag_counting_engine import PosTagCounter
from blabla.utils.exceptions import *
import blabla.utils.settings as settings
from blabla.utils.global_params import *
import os
import stanza
import nltk
import numpy as np
import math
import json

## Ref: http://www.surdeanu.info/mihai/teaching/ista555-fall13/readings/PennTreebankConstituents.html
class Sentence(object):
	"""The class that is responsible for processing a sentence and extracting the fundamental blocks such as
		the dependency parse tree, constituency parse tree and other low level features for aggregation later.
	"""

	def __init__(self, language, nlp, client, raw_text, sent_json=None):
		"""The initialization method for the Sentence class
		Args:
			language (str): The language of the raw_text
			nlp (obj): The Stanza NLP object.
			client (obj): The CoreNLP client object.
			raw_text (str): The raw text 
			sent_json (JSON): The json string of a sentence if the input is a JSON
		Returns:
			None:
		"""
		self.lang = language
		self._pos_tag_counter = None
		self._sent = raw_text
		self._stanza_doc = None
		self._const_pt = None
		self._yngve_tree_root = None
		self.nlp = nlp
		self.client = client
		self.json = sent_json

	@property
	def pos_tag_counter(self):
		"""A property method to return the part of speech counter object"""
		return self._pos_tag_counter

	@property
	def sent(self):
		"""A property method to return the raw text of the current sentence object"""
		return self._sent

	@property
	def stanza_doc(self):
		"""A property method to return the Stanza object"""
		return self._stanza_doc

	@property
	def yngve_tree_root(self):
		"""A property method to return the root node of the Yngve tree"""
		return self._yngve_tree_root

	@property
	def const_pt(self):
		"""A property method to return the constituency parse tree"""
		return self._const_pt

	@property
	def tot_num_characters(self):
		"""A property method to return the total number of characters in the current sentence"""
		return math.sum([len(word) for word in self._stanza_doc.sentences[0].words])

	@property
	def speech_time(self):
		"""A property method to return the speech time"""
		return math.fsum([word['duration'] for word in self.json['words']])

	@property
	def start_time(self):
		"""A property method to return the start time of the current sentence"""
		return self.json['words'][0]['start_time']

	@property
	def end_time(self):
		"""A property method to return the end time of the current sentence"""
		return self.json['words'][-1]['start_time'] + self.json['words'][-1]['duration']

	@property
	def locution_time(self):
		"""A property method to return the locution time of the current sentence"""
		start_time = self.start_time
		end_time = self.end_time
		return end_time - start_time

	@property
	def words_per_min(self):
		"""A property method to return the words per min for the current sentence"""
		return (self.num_words() / self.locution_time()) * 60.0

	def _get_gaps(self):
		gaps = []
		for prev_word, next_word in zip(self.json['words'][:-1], self.json['words'][1:]):
			prev_end_time = prev_word['start_time'] + prev_word['duration']
			gaps.append(next_word['start_time'] - prev_end_time)
		return gaps

	def num_pauses(self, pause_duration):
		"""A method to calculate the number of pauses in the current sentence
			Args:
				pause_duration (float): The value for the duration of the pause
			Returns:
				int: The number of pauses
		"""
		gaps = self._get_gaps()     
		return sum(gap > pause_duration for gap in gaps)

	def tot_pause_time(self, pause_duration):
		"""A method to calculate the total pause time in the current sentence
			Args:
				pause_duration (float): The value for the duration of the pause
			Returns:
				float: The total pause time of the sentence
		"""
		return sum([gap for gap in self._get_gaps()  if gap > pause_duration])

	def analyze_text(self):
		"""A method to get the Stanza document
			Args:
				None: 
			Returns:
				doc (Stanza): The Stanza document object
		"""
		doc = self.nlp(self._sent)
		return doc

	def const_parse_tree(self):
		"""A method to get the constituency parse tree
			Args:
				None: 
			Returns:
				parseTree (CoreNLP): The CoreNLP constituency parse tree object
		"""
		document = self.client.annotate(self._sent)
		document = json.loads(document.text)
		pt = document["sentences"][0]["parse"]
		parseTree = Tree.fromstring(pt)
		return parseTree

	def num_words(self):
		"""A method to get the number of words
			Args:
				None: 
			Returns:
				int: The number of words in the current sentence
		"""
		return len(self._stanza_doc.sentences[0].words)

	def _navigate_and_score_leaves(self, yngve_node, score_so_far):
		"""A method to assign Yngve scores to the leaf nodes
			Args:
				None: 
			Returns:
				Yngve_Tree: The Yngve tree with the scores for the leaf nodes
		"""
		if len(yngve_node.children) == 0:
			yngve_node.score = score_so_far
		else:  # it has child nodes
			for child in yngve_node.children:
				self._navigate_and_score_leaves(child, score_so_far + child.score)

	def _traverse_and_build_yngve_tree(self, start_node, parent_node):
		"""A method to construct the Yngve tree
			Args:
				None: 
			Returns:
				Yngve_Tree: The Yngve tree constructed from the constituency parse tree
		"""
		score = 0
		for child in start_node[::-1]:
			if isinstance(child, str):
				curr_node = YngveNode(child, 0, parent_node)
			elif isinstance(child, nltk.tree.Tree):
				curr_node = YngveNode(child.label(), score, parent_node)
				score += 1
				self._traverse_and_build_yngve_tree(child, curr_node)

	def yngve_tree(self):
		"""The main method to construct the Yngve tree and assign values to all leaf nodes
			Args:
				None
			Returns:
				yngve_tree_root_node (YngveNode): The root node of the yngve tree
		"""
		sent_child = self._const_pt[0]
		yngve_tree_root_node = YngveNode("S", 0)
		self._traverse_and_build_yngve_tree(sent_child, yngve_tree_root_node)
		self._navigate_and_score_leaves(
			yngve_tree_root_node, yngve_tree_root_node.score
		)
		return yngve_tree_root_node

	def setup_dep_pt(self):
		"""A method to construct the dependency parse tree
			Args:
				None
			Returns:
				None:
		"""
		if len(self._sent) == 0:
			raise EmptyStringException('The input string is empty')

		try:
			self._stanza_doc = self.analyze_text()
		except Exception as e:
			raise DependencyParsingTreeException('Dependency parse tree set up failed')

		try:
			self._pos_tag_counter = PosTagCounter(self._stanza_doc)
		except Exception as e:
			raise POSTagExtractionFailedException('POS Tag counter failed')

	def setup_const_pt(self):
		"""A method to construct the constituency parse tree
			Args:
				None
			Returns:
				None:
		"""
		try:
			self._const_pt = self.const_parse_tree()
		except Exception as e:
			raise ConstituencyTreeParsingException(
				'Constituency parse tree set up failed. Please check if the input format (json/string) is mentioned correctly'
			)

	def setup_yngve_tree(self):
		"""A method to construct the Yngve tree
			Args:
				None
			Returns:
				None:
		"""
		try:
			self._yngve_tree_root = self.yngve_tree()
		except Exception as e:
			raise YngveTreeConstructionException('Yngve tree set up failed')

## (4) Dependency Parse

Please see the Spacy Tutorial: 

https://spacy.io/usage/linguistic-features#dependency-parse

## (5) Word2vec features (representation)

(1) Exploring Word2Vec:

https://github.com/lutzhamel/word2vec-simplified/blob/master/Exploring%20Word2Vec.ipynb

(2) Tutorials

https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py